In [4]:
import sys
import os

print("main module loaded")
print("sys.path before:", sys.path)

# Determine the script directory
script_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()

# Add src directory to sys.path
src_path = os.path.join(script_dir, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print("sys.path after:", sys.path)

from src.data_loader import load_data
from src.text_splitter import split_text
from src.embeddings import get_embeddings
from src.vector_store import create_vector_store
from src.retriever import get_retriever
from src.pipeline import generate_answer
from src.hybrid_search import hybrid_search

print("All imports successful")

dataset_name = "databricks/databricks-dolly-15k"
page_content_column = "context"

data = load_data(dataset_name, page_content_column)
docs = split_text(data)
embeddings = get_embeddings()
db = create_vector_store(docs, embeddings)
retriever = get_retriever(db)

question = "Where is Indonesia Located?"
hybrid_results = hybrid_search(retriever, db, question)
answer = generate_answer(hybrid_results, question)
print('Answer: ', answer)

main module loaded
sys.path before: ['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2288.0_x64__qbz5n2kfra8p0', 'c:\\Code\\Machine Learning - Workspace\\Avalon\\Final-Project-Avalon\\.venv', '', 'c:\\Code\\Machine Learning - Workspace\\Avalon\\Final-Project-Avalon\\.venv\\Lib\\site-packages', 'c:\\Code\\Machine Learning - Workspace\\Avalon\\Final-Project-Avalon\\.venv\\Lib\\site-packages\\win32', 'c:\\Code\\Machine Learning - Workspace\\Avalon\\Final-Project-Avalon\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Code\\Machine Learning - Workspace\\Avalon\\Final-Project-Avalon\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Code\\Machi

ImportError: cannot import name 'create_vector_store' from 'src.vector_store' (c:\Code\Machine Learning - Workspace\Avalon\Final-Project-Avalon\src\vector_store.py)